### COMP - 472
Assignment Two

AI GURUS: James Partsafas, Ghaith Chrit, Samuel Collette

---------

#### Task 1

1) Add imports

In [179]:
import os
import gc
import csv
import random
import nltk 
import matplotlib.pyplot as plt
import pandas as pd
import gensim.downloader as api
from gensim.models import KeyedVectors, Word2Vec
from urllib import request
print("Imports complete")

Imports complete


2) Load model

In [180]:
print("starting model load")
model_name = 'word2vec-google-news-300'
model = KeyedVectors.load_word2vec_format(api.load(model_name, return_path=True), binary=True) 
print("model loaded")

starting model load
model loaded


3) Load test dataset

In [181]:
dataset_path = 'A2-DataSet/synonym.csv'
df = pd.read_csv(dataset_path)
print("Dataset loaded")

Dataset loaded


4) Run the model on test data and report result

In [182]:
if (not ('model_name' in locals())):
    model_name = ""
details_output_file = os.path.join('output', f'{model_name}-details.csv')
analysis_output_file = os.path.join('output', f'analysis.csv')

In [183]:
def create_details_file(details_output_file_name, df, model):
    correct_labels = 0
    answered_questions = 0
    with open(details_output_file_name, mode='w', newline='') as details_file:
      details_writer = csv.writer(details_file)
      details_writer.writerow(['question-word', 'correct-answer', 'guess-word', 'label'])

      for _, row in df.iterrows():
        question_word, correct_answer, guess_words = row['question'], row['answer'], row[2:].to_list()
        guess_words_in_vocab = [word for word in guess_words if word in model.index_to_key]

        if question_word in model.index_to_key and len(guess_words_in_vocab) > 0:
          most_similar_word = model.most_similar_to_given(question_word, guess_words_in_vocab)
          label = 'correct' if most_similar_word == correct_answer else 'wrong'
          correct_labels += 1 if label == 'correct' else 0
          answered_questions += 1
        else:
          label = 'guess'
          most_similar_word = random.choice(guess_words)

        details_writer.writerow([question_word, correct_answer, most_similar_word, label])
    
    return (correct_labels, answered_questions)

def write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, write_mode):
    accuracy = correct_labels / answered_questions if answered_questions > 0 else 0

    with open(analysis_output_file, mode=write_mode, newline='') as analysis_file:
      analysis_writer = csv.writer(analysis_file)
      if (write_mode == 'w'):  
          analysis_writer.writerow(['Model Name', 'Vocab Length', 'Number of Correct Labels', 'Number of Answered Questions', 'Accuracy of Answered Questions'])
      analysis_writer.writerow([model_name, len(model.index_to_key), correct_labels, answered_questions, accuracy])

In [184]:
correct_labels, answered_questions = create_details_file(details_output_file, df, model)
print(correct_labels, answered_questions)

write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'w')

70 79


In [185]:
# Rid model to clear memory
model = None
gc.collect()

23550

------------
#### Task 2

In [186]:
# First of 2 models with same embedding size
print("starting model load")
model_name = 'glove-twitter-50'
model = api.load(model_name)
print("model loaded")

starting model load
model loaded


In [187]:
details_output_file = os.path.join('output', f'{model_name}-details.csv')
correct_labels, answered_questions = create_details_file(details_output_file, df, model)
print(correct_labels, answered_questions)

write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'a')

36 78


In [188]:
model = None
gc.collect()

13

In [189]:
# Second of 2 models with same embedding size
print("starting model load")
model_name = 'glove-wiki-gigaword-50'
model = api.load(model_name)
print("model loaded")

starting model load
model loaded


In [190]:
details_output_file = os.path.join('output', f'{model_name}-details.csv')
correct_labels, answered_questions = create_details_file(details_output_file, df, model)
print(correct_labels, answered_questions)

write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'a')

57 80


In [191]:
model = None
gc.collect()

13

In [192]:
# First of 2 models with different embedding size
print("starting model load")
model_name = 'glove-twitter-25'
model = api.load(model_name)
print("model loaded")

starting model load
model loaded


In [193]:
details_output_file = os.path.join('output', f'{model_name}-details.csv')
correct_labels, answered_questions = create_details_file(details_output_file, df, model)
print(correct_labels, answered_questions)

write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'a')

36 78


In [194]:
model = None
gc.collect()

13

In [195]:
# Second of 2 models with different embedding size
print("starting model load")
model_name = 'glove-wiki-gigaword-100'
model = api.load(model_name)
print("model loaded")

starting model load
model loaded


In [196]:
details_output_file = os.path.join('output', f'{model_name}-details.csv')
correct_labels, answered_questions = create_details_file(details_output_file, df, model)
print(correct_labels, answered_questions)

write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'a')

65 80


In [197]:
model = None
gc.collect()

13

In [198]:
def create_accuracy_bar_chart(models,outputFile):
    analysis_file = 'output/analysis.csv'
    df_analysis = pd.read_csv(analysis_file)
    analysis_rows = df_analysis.loc[df_analysis['Model Name'].isin(models)]
    
    students_file = 'A2-DataSet/COMP-472-per-question.csv'
    df_students = pd.read_csv(students_file, encoding='UTF-16 LE')
    students_average_row = df_students.loc[0]
    
    accuracy_data = {'Baseline': 50, 'Students': students_average_row['Accuracy']}
    for _, row in analysis_rows.iterrows():
        accuracy_data[row['Model Name']] = row['Accuracy of Answered Questions'] * 100
        
    keys = list(accuracy_data.keys())
    values = list(accuracy_data.values())
    
    fig, ax = plt.subplots()
    plt.bar(keys, values)
    ax.tick_params(axis='x', labelrotation = 30)
    plt.gcf().subplots_adjust(bottom=0.4)
    
    plt.xlabel('Models')
    plt.ylabel('Accuracy (%)')
    plt.title('Accuracy of different models')
    
    plt.savefig(outputFile)
    plt.close(fig)

In [199]:
create_accuracy_bar_chart(['glove-twitter-50', 'glove-wiki-gigaword-50', 'glove-twitter-25', 'glove-wiki-gigaword-100'],'output/task2_accuracy.png')

------------
#### Task 3

Creating necessary functions

In [200]:
def preprocess(url) -> list:
    response = request.urlopen(url)
    raw=response.read().decode('utf8')
    tokens_sen=nltk.sent_tokenize(raw)
    i=0
    for sen in tokens_sen:
        words= nltk.word_tokenize(sen)
        tokens_sen[i]=words
        i+=1
    return tokens_sen

def model_creation_and_output(modName,tokens,windowSizes:list, vectorSizes:list):
    i = 0
    global model,model_name
    while i < len(vectorSizes):
        j=0
        while j < len(windowSizes):
            print("starting model load")
            model_name = f'word2vec-{modName}-{vectorSizes[i]}-{windowSizes[j]}'
            model = Word2Vec(sentences = tokens, window = windowSizes[j], vector_size=vectorSizes[i])
            word_vectors = model.wv
            word_vectors.save('vectors.kv')
            model = KeyedVectors.load('vectors.kv')
            print("model loaded")

            details_output_file = os.path.join('output', f'{model_name}-details.csv')
            correct_labels, answered_questions = create_details_file(details_output_file, df, model)
            print(correct_labels, answered_questions)
            
            write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'a')
            j+=1
        i+=1

def multi_bar_chart(vectorSizes,windowSizes,names):
    
    model_name_list=[]
    
    i=0
    while i < len(vectorSizes):
        j=0
        while j < len(windowSizes):
            for name in names:
                model_name_list.append(f'word2vec-{name}-{vectorSizes[i]}-{windowSizes[j]}') 
            create_accuracy_bar_chart(model_name_list, f'output/task3_accuracy-{vectorSizes[i]}-{windowSizes[j]}.png')
            model_name_list=[]
            j+=1
        i+=1

Creating variables, and preprocesing the books

In [201]:
url1 = "https://www.gutenberg.org/cache/epub/2600/pg2600.txt" #war-and-peace
url2 = "https://www.gutenberg.org/cache/epub/28054/pg28054.txt" #brothers-karamazov
url3 = "https://www.gutenberg.org/cache/epub/2554/pg2554.txt" #crime-and-punishment
url4 = "https://www.gutenberg.org/cache/epub/7178/pg7178.txt" #Swann's-Way
url5 = "https://www.gutenberg.org/cache/epub/1399/pg1399.txt" #Anna-Karenina
  
tokens_url1 = preprocess(url1) 
tokens_url2 = preprocess(url2) 
tokens_url3 = preprocess(url3) 
tokens_url4 = preprocess(url4) 
tokens_url5 = preprocess(url5) 

window_size = [5,3]

vector_size = [100,50]

names = ["war-and-peace","brothers-karamazov","crime-and-punishment","Swann's-Way","Anna-Karenina"]


Creating models, writing output to files, and creating bar charts to compare easily

In [202]:
model_creation_and_output("war-and-peace",tokens_url1,window_size,vector_size)
model_creation_and_output("brothers-karamazov",tokens_url2,window_size,vector_size)
model_creation_and_output("crime-and-punishment",tokens_url3,window_size,vector_size)
model_creation_and_output("Swann's-Way",tokens_url4,window_size,vector_size)
model_creation_and_output("Anna-Karenina",tokens_url5,window_size,vector_size)

multi_bar_chart(vector_size,window_size,names)

starting model load
model loaded
8 29
starting model load
model loaded
7 29
starting model load
model loaded
7 29
starting model load
model loaded
5 29
starting model load
model loaded
10 23
starting model load
model loaded
9 23
starting model load
model loaded
8 23
starting model load
model loaded
8 23
starting model load
model loaded
7 16
starting model load
model loaded
8 16
starting model load
model loaded
9 16
starting model load
model loaded
6 16
starting model load
model loaded
7 19
starting model load
model loaded
7 19
starting model load
model loaded
9 19
starting model load
model loaded
7 19
starting model load
model loaded
7 21
starting model load
model loaded
4 21
starting model load
model loaded
7 21
starting model load
model loaded
5 21
